### CUSTOM LAYERS

+ Lets say you have a Sequence of Layers like `A B C | A B C | A B C | A B C `
+ So here you may want to create a layer D contains A B C.
+ And you can repeat the D layer four times.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
## fetch the data.
housing = fetch_california_housing()
# housing.data
# housing.target
print(housing.keys())
print()
print("====================================Complete Dataset======================================")
## X_train_full, X_test, y_train_full, y_test
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1,1), random_state = 42)
print(X_train_full.shape)
print(X_test.shape)
print(y_train_full.shape)
print(y_test.shape)
print()
print("====================================Dataset After Splitting to Validation Set======================================")
## X_valid, X_test, y_valid, y_test
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)
print()

print("=================================Standardize the Data==============================================")
## Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
print(X_train_scaled.shape)
X_valid_scaled = scaler.transform(X_valid)
print(X_valid_scaled.shape)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled.shape)

## defining the input shape
input_shape = X_train.shape[1:]
print(input_shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

====================================Complete Dataset======================================
(15480, 8)
(5160, 8)
(15480, 1)
(5160, 1)

====================================Dataset After Splitting to Validation Set======================================
(11610, 8)
(3870, 8)
(11610, 1)
(3870, 1)

=================================Standardize the Data==============================================
(11610, 8)
(3870, 8)
(5160, 8)
(8,)


In [4]:
## defining the huber function
## this is the loss function
def huber_fn(y_true, y_pred):
    ## define the error from y_true and y_pred, that will be the difference
    error = y_true - y_pred
    #print(error)
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn


In [7]:
## wxponntial layer
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))
print(exponential_layer([-1., 0., 1.]))

tf.Tensor([0.36787945 1.         2.7182817 ], shape=(3,), dtype=float32)


In [8]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=input_shape),
    keras.layers.Dense(1),
    exponential_layer
])
model.compile(loss="mse", optimizer="sgd")
model.fit(X_train_scaled, y_train, epochs=5,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/5
363/363 [==============================] - 1s 2ms/step - loss: 0.9047 - val_loss: 0.4006
Epoch 2/5
363/363 [==============================] - 1s 2ms/step - loss: 0.5408 - val_loss: 0.3818
Epoch 3/5
363/363 [==============================] - 1s 2ms/step - loss: 0.4111 - val_loss: 0.3564
Epoch 4/5
363/363 [==============================] - 1s 1ms/step - loss: 0.3965 - val_loss: 0.3520
Epoch 5/5
162/162 [==============================] - 0s 922us/step - loss: 0.4054


0.40536990761756897

In [12]:
## creaying the dense layer
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

## model development    
model = keras.models.Sequential([
    MyDense(30, activation="relu", input_shape=input_shape),
    MyDense(1)
])

## compile the model
model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)
model.save("my_model_with_a_custom_layer.h5")

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 1.6129 - val_loss: 5.0784
Epoch 2/2
162/162 [==============================] - 0s 1ms/step - loss: 0.4794


In [13]:
## when loading the model again
## specify the dense layer
model = keras.models.load_model("my_model_with_a_custom_layer.h5",
                                custom_objects={"MyDense": MyDense})

In [15]:
## to create layer with multple inouts
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        print("X1.shape: ", X1.shape ," X2.shape: ", X2.shape) # Debugging of custom layer
        return X1 + X2, X1 * X2

    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]
    
inputs1 = keras.layers.Input(shape=[2])
inputs2 = keras.layers.Input(shape=[2])
outputs1, outputs2 = MyMultiLayer()((inputs1, inputs2))

X1.shape:  (None, 2)  X2.shape:  (None, 2)


In [16]:
def split_data(data):
    columns_count = data.shape[-1]
    half = columns_count // 2
    return data[:, :half], data[:, half:]

X_train_scaled_A, X_train_scaled_B = split_data(X_train_scaled)
X_valid_scaled_A, X_valid_scaled_B = split_data(X_valid_scaled)
X_test_scaled_A, X_test_scaled_B = split_data(X_test_scaled)

# Printing the splitted data shapes
X_train_scaled_A.shape, X_train_scaled_B.shape

((11610, 4), (11610, 4))

In [17]:
outputs1, outputs2 = MyMultiLayer()((X_train_scaled_A, X_train_scaled_B))

X1.shape:  (11610, 4)  X2.shape:  (11610, 4)


In [18]:
input_A = keras.layers.Input(shape=X_train_scaled_A.shape[-1])
input_B = keras.layers.Input(shape=X_train_scaled_B.shape[-1])
hidden_A, hidden_B = MyMultiLayer()((input_A, input_B))
hidden_A = keras.layers.Dense(30, activation='selu')(hidden_A)
hidden_B = keras.layers.Dense(30, activation='selu')(hidden_B)
concat = keras.layers.Concatenate()((hidden_A, hidden_B))
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

X1.shape:  (None, 4)  X2.shape:  (None, 4)


In [19]:
model.compile(loss='mse', optimizer='nadam')

In [20]:
model.fit((X_train_scaled_A, X_train_scaled_B), y_train, epochs=2,
          validation_data=((X_valid_scaled_A, X_valid_scaled_B), y_valid))

Epoch 1/2
X1.shape:  (None, 4)  X2.shape:  (None, 4)
X1.shape:  (None, 4)  X2.shape:  (None, 4)
363/363 [==============================] - 1s 2ms/step - loss: 1.7667 - val_loss: 1.7146
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.9727 - val_loss: 1.1123


In [22]:
## Now let's create a layer with a different behavior during training and testing:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape
    
model = keras.models.Sequential([
    AddGaussianNoise(stddev=1.0),
    keras.layers.Dense(30, activation="selu"),
    keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
363/363 [==============================] - 1s 2ms/step - loss: 2.3508 - val_loss: 0.8232
Epoch 2/2
162/162 [==============================] - 0s 1ms/step - loss: 0.7630


0.762959897518158